以MATH数据集为例,给出TOT的实现方案.

In [1]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import re
import sys
from collections import defaultdict, deque
from typing import List
import re
from typing import Any, List
import ipydagred3
import networkx as nx
import numpy as np
import openai
from tqdm import tqdm
from utils import *

#os.environ["http_proxy"] = "http://localhost:7890"
#os.environ["https_proxy"] = "http://localhost:7890"
setOpenAi(keyid = 4)

GPT_MODEL = "gpt-4-turbo"
GPT_MODEL = "gpt-4o-mini"
#GPT_MODEL = "gpt-3.5-turbo"

# 读取问题数据文件
file_path = 'math200.json'
N = 100
with open(file_path, 'r', encoding='utf-8') as file:
    problems = json.load(file)

In [2]:
def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1
    
    if right_brace_idx == None:
        retval = None
    else:
        retval = string[idx:right_brace_idx + 1]
    
    return retval


def remove_boxed(s):
    left = "\\boxed{"
    try:
        assert s[:len(left)] == left
        assert s[-1] == "}"
        return s[len(left):-1]
    except:
        return None

In [3]:
file_path = 'math200.json'
N = 100
count = 0
questions = []
types = []
gold_answers = []
# 打开文件并逐行读取内容
with open(file_path, 'r', encoding='utf-8') as file:
    dataset = json.load(file)

for item in dataset:
    questions.append(item['problem'])
    types.append(item['type'])
    gold_answers.append(item['solution'])
    count += 1
    if count >= N:
        break


In [4]:
question_id = 2

question = problems[question_id]['problem']
type = problems[question_id]['type']
gold_answer = answer = remove_boxed(last_boxed_only_string(problems[question_id]['solution']))

In [5]:
gold_answer

'8\\sqrt{3} \\text { square inches}'

In [6]:
problems[question_id]['solution']

'The two triangles make a smaller hexagon inside the large hexagon with the same center. Draw six lines from the center to each of the vertices of the small hexagon. Both triangles are now divided into $9$ congruent equilateral triangles, with the smaller hexagon region taking $\\frac69=\\frac23$ of the triangle.\n\nThe triangle is $\\frac12$ of the larger hexagon, so the smaller hexagon is $\\frac12 \\cdot \\frac23 = \\frac13$ of the larger hexagon.\n\nWe now find the area of the large hexagon. By drawing six lines from the center to each of the vertices, we divide the hexagon into six equilateral triangles with side length $4$. The area of an equilateral triangle with side length $s$ is $\\frac{s^2 \\cdot \\sqrt{3}}{4}$, so the area of each triangle is $\\frac{16 \\sqrt{3}}{4}=4\\sqrt{3}$. Therefore, the area of the large hexagon is $24 \\sqrt{3}$. The area of the smaller hexagon, which is the region common to the two triangles, is $\\frac13 \\cdot 24 \\sqrt3=\\boxed{8\\sqrt{3} \\tex

In [7]:
question

'By joining alternate vertices of a regular hexagon with edges $4$ inches long, two equilateral triangles are formed, as shown. What is the area, in square inches, of the region that is common to the two triangles? Express your answer in simplest radical form. [asy]\ndraw((0,3)--(0,8)--(4,11)--(8,8)--(8,3)--(4,0)--cycle,black+linewidth(1));\ndraw((4,0)--(0,8)--(8,8)--cycle, black+dashed+linewidth(1));\ndraw((0,3)--(4,11)--(8,3)--cycle, black+dotted+linewidth(1));\nlabel("4",(8,5.5),E);\n[/asy]'

In [8]:
type

'Geometry'

任务分解

- scy的decompose_sql 只有一个output
- 第二种我改的是有两个output， 改function

In [18]:
def decompose_sql(question, type):   

    prompt_for_decompose = f"""I will now give you a math problem. The type of problem is {type}. Please help me translate this math problem into a series of step-by-step sub-problems.

1 examples are as follows:
Question: Four years ago, Kody was only half as old as Mohamed. If Mohamed is currently twice 30 years old, how old is Kody? 
Answer: 
To solve the question "How old is Kody?", we need to solve the following problems step by step:
1. How old is Mohamed now?
2. How old was Mohamed four years ago?
3. How old was Kody four years ago?
4. How old is Kody now?

Now the command is {question}, please decompose it into a series of easy-to-solve steps like the examples.
Answer Format: (Please write each broken-down question step on a separate line, starting with a number.)
To solve the question "xxx", we need to solve the following problems step by step:
1. sub-question 1
2. sub-question 2
3. sub-question 3
...

"""

    Q = {
        "role": "user",
        "content": prompt_for_decompose
    }
    # Query = Example+[Q]
    Query = [Q]
    result = askChatGPT(Query, model=GPT_MODEL, temperature=1)
    return result  , Query


def convert_steps_to_format(decom_commands):
    # 截取“we need to know:”后的内容
    start_index = decom_commands.find("we need to solve the following problems step by step:") + len("we need to solve the following problems step by step:")
    subtasks_text = decom_commands[start_index:].strip()
    # 将每个子任务单独列出
    subtasks = subtasks_text.split('\n')
    subtasks = [task.strip().split('. ', 1)[-1] for task in subtasks]
    steps_dict = {index: value for index, value in enumerate(subtasks)}
    return subtasks, steps_dict

# decompose_steps = decompose_sql(question, type)
# steps, steps_dict = convert_steps_to_format(decompose_steps)
# num_steps = len(steps)
# print(steps)
# print(steps_dict)  # 只是加了一个问题的编号而已.

In [11]:
print(len(steps))

7


In [12]:
# TOT 求解
N = 3  # 每个子问题进行N次proposal
M = 2  # 通过评估选出M个最好的proposal

solution = []

for i in range(num_steps):
    subtask = steps[i]
    sys_q = f"""
There is a math_problem. I need you to solve it and give an answer.
Here is the problem:\n{question}

I have broken this math problem down into a series of smaller problems. I will assign you sub-problems one by one, and provide the results of the previous sub-problems as a reference for your reasoning.
Please solve the problem and respond according to mathematical logic.
"""  

    subask = f"""\nThe sub-problem to solve now is xxx: {subtask}
Based on the information above, please provide a concise and clear answer"""
    
    if len(solution)==0:
        # 第一个子问题
        query = subask
        Q = [{'role':'system', 'content':sys_q},
            {'role':'user', 'content':query},]
        for n in range(N):  # 一个子问题提问N次,获取N个解
            result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
            eval_Q = Q + [{'role':'assistant', 'content':result}]
            eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
            score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
            score = int(score)
            
            solution.append((score, [result]))  # 维护一整条推理路径
            
        solution = sorted(solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝
    else:
        temp_solution = []
        for m in range(M):  # 因为剪枝动态维护M个推理路径
            answersSoFar = f"""\nSo far, the answers to the preceding sub-problems are as follows: The format is Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx."""
            for index, value in enumerate(solution[m][1]):
                try:
                    answersSoFar += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""
                except:
                    print('warning')
                    print(index)
                    print(len(solution[m][1]))
                    print(len(steps))
                    sys.exit(0)
            query = answersSoFar+subask
            Q = [{'role':'system', 'content':sys_q},
                 {'role':'user', 'content':query},]
            for n in range(N):  # 一个子问题提问N次,获取N个解
                result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
                eval_Q = Q + [{'role':'assistant', 'content':result}]
                eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
                score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
                score = int(score)
                
                temp_solution.append((solution[m][0]+score, solution[m][1]+[result]))  # 路径score累加
        
        # print(len(temp_solution))  # 此时temp_solution中应该有M*N种推理路径
        solution = sorted(temp_solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝 M*N->M

print(len(solution))
printSeq(solution)
# 用额外的一次query再问一下最终的答案


total tokens: 330
total tokens: 366
total tokens: 322
total tokens: 358
total tokens: 323
total tokens: 359
total tokens: 541
total tokens: 577
total tokens: 529
total tokens: 565
total tokens: 529
total tokens: 565
total tokens: 522
total tokens: 558
total tokens: 517
total tokens: 553
total tokens: 526
total tokens: 562
total tokens: 776
total tokens: 812
total tokens: 785
total tokens: 821
total tokens: 781
total tokens: 817
total tokens: 734
total tokens: 770
total tokens: 715
total tokens: 751
total tokens: 719
total tokens: 755
total tokens: 1034
total tokens: 1070
total tokens: 1021
total tokens: 1057
total tokens: 1034
total tokens: 1070
total tokens: 1049
total tokens: 1085
total tokens: 1092
total tokens: 1128
total tokens: 1066
total tokens: 1102
total tokens: 1162
total tokens: 1198
total tokens: 1138
total tokens: 1174
total tokens: 1150
total tokens: 1186
total tokens: 1136
total tokens: 1172
total tokens: 1118
total tokens: 1154
total tokens: 1114
total tokens: 1150
tota

我刚刚看了一下tot，他每个问题的过程的方法好像都不太一样，他那个写作是对于任务分解做N次，然后选M个最好的，然后再根据选出来的plan推理N次，最后得出一个最好的。

In [19]:
# TOT 第二种求解方式

N = 3  # 先进性N次任务分解，然后 N次做题，最后评估


# 任务分解阶段
decompositions = []
for i in range(N):
    plan, Q = decompose_sql(question, type)
    result, result_dict = convert_steps_to_format(plan)
    
    eval_Q = Q + [{'role':'assistant', 'content':f"""The plan is : {result}"""}]
    eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this step to step plan, on a scale from 1 to 10. Only output the number."}]
    printSeq(eval_Q)
    score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
    score = int(score)
    
    decompositions.append((score, result_dict))

# 选择最佳任务分解方案
task_decompositions = sorted(decompositions, key=lambda x: x[0], reverse=True)
best_task_decomposition = task_decompositions[0][1] 

print("******************************解题阶段******************************")
# 解题阶段
solutions = []
for i in range(N):
    question_input = f"""
There is a math_problem. I need you to solve it and give an answer.
Here is the problem:\n{question}

I have broken this math problem down into a series of smaller problems. Here is the list of sub-problems: {best_task_decomposition}
Please solve the problem and respond according to mathematical logic.
    
    """
    query = {"role": "user", "content": question_input}
    Q_jie = [query]
    sol = askChatGPT(Q_jie, model=GPT_MODEL, temperature=1)
    sol_eval_Q = Q_jie + [{'role':'assistant', 'content':sol}]
    sol_eval_Q = sol_eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 10. Only output the number."}]
    printSeq(sol_eval_Q)
    sol_score = askChatGPT(sol_eval_Q, model=GPT_MODEL, temperature=1)
    sol_score = int(sol_score)
    
    solutions.append((sol_score, sol))

# 选择最佳解答
sub_solutions = sorted(solutions, key=lambda x: x[0], reverse=True)
best_solution = sub_solutions[0][1]  # 选择最好的子问题解

print("best_solution: ", best_solution)

print("******************************评估结果******************************")

last_query = [{
    "role": "user",
    "content": f""" 
    Given the problem and step-by-step plan and the solution as following: {best_solution}
    
    What is the final answer?
    Please give the final answer without any additional explanation or clarification."""
}]

finalResult = askChatGPT(last_query, model=GPT_MODEL, temperature=1)
print(finalResult)


# 让大语言模型来判断有没有回答正确
judgeAnswer = {'role':'user', 'content':f"""Here is a math problem with a standard answer and a student's solution. Please help me determine if the student's solution is correct. If the numerical value are same, then it is correct.

Problem: {question}

Standard answer: {gold_answer}

Answer: {finalResult}

If the student's answer is correct, just output True; otherwise, just output False.
No explanation is required.
"""}

Q_judge = [judgeAnswer]
ifcorrect = askChatGPT(Q_judge, model=GPT_MODEL, temperature=1)  # 要么是True, 要么是False

if 'True' in ifcorrect:
    print('correct')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功
elif 'False' in ifcorrect:
    print('error')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功                       


total tokens: 546
{'role': 'user', 'content': 'I will now give you a math problem. The type of problem is Geometry. Please help me translate this math problem into a series of step-by-step sub-problems.\n\n1 examples are as follows:\nQuestion: Four years ago, Kody was only half as old as Mohamed. If Mohamed is currently twice 30 years old, how old is Kody? \nAnswer: \nTo solve the question "How old is Kody?", we need to solve the following problems step by step:\n1. How old is Mohamed now?\n2. How old was Mohamed four years ago?\n3. How old was Kody four years ago?\n4. How old is Kody now?\n\nNow the command is By joining alternate vertices of a regular hexagon with edges $4$ inches long, two equilateral triangles are formed, as shown. What is the area, in square inches, of the region that is common to the two triangles? Express your answer in simplest radical form. [asy]\ndraw((0,3)--(0,8)--(4,11)--(8,8)--(8,3)--(4,0)--cycle,black+linewidth(1));\ndraw((4,0)--(0,8)--(8,8)--cycle, black

In [20]:
print(finalResult)

\(\boxed{4\sqrt{3}}\) square inches.


In [50]:
print(gold_answer)

8\sqrt{3} \text { square inches}


In [15]:
printSeq(sub_solutions)

NameError: name 'sub_solutions' is not defined

In [20]:
finalResult

'The final answer is (-1 + √5)/2, (1 + √5)/2 and (-1 - √5)/2, (1 - √5)/2.'

In [21]:
gold_answer

'To find the $x$-coordinates of the intersections, substitute $x^2$ for $y$ in $x+y=1$ and solve for $x$, resulting in  \\begin{align*}\nx+x^2&=1 \\\\\n\\Rightarrow \\qquad x^2+x-1&=0 \\\\\n\\Rightarrow \\qquad x&=\\frac{-1\\pm\\sqrt{1+4}}2=\\frac{-1\\pm\\sqrt5}2\\\\\n\\end{align*}Using each of these coordinates to solve for $y$ gives us the intersections at $\\left(\\frac{-1+\\sqrt5}2,\\frac{3-\\sqrt5}2\\right)$ and $\\left(\\frac{-1-\\sqrt5}2,\\frac{3+\\sqrt5}2\\right)$.  Using the distance formula, we have \\begin{align*}\n&\\sqrt{ \\left(\\frac{-1+\\sqrt5}{2}-\\frac{-1-\\sqrt5}{2}\\right)^2 + \\left(\\frac{3-\\sqrt5}2-\\frac{3+\\sqrt5}2\\right)^2 }\\\\\n&\\qquad=\\sqrt{\\left(\\frac{2\\sqrt5}2\\right)^2 + \\left(-\\frac{2\\sqrt5}2\\right)^2}\\\\\n&\\qquad=\\sqrt{ 2\\sqrt5^2 }\\\\\n&\\qquad=\\boxed{\\sqrt{10}}.\n\\end{align*}'

In [ ]:
solution

[(9,
  ['To find the intersection points of the two equations $y=x^2$ and $x+y=1$, we can substitute $y=x^2$ into the second equation to get:\n$x + x^2 = 1$\nwhich simplifies to:\n$x^2 + x - 1 = 0$\n\nSolving this quadratic equation will give us the x-coordinates of the intersection points.',
   'To find the intersection points, we need to solve the quadratic equation $x^2 + x - 1 = 0$. By using the quadratic formula, we find that the solutions are $x = \\frac{-1 \\pm \\sqrt{5}}{2}$.\n\nNow, we need to calculate the distance between the two intersection points. Let the intersection points be $A\\left(\\frac{-1 + \\sqrt{5}}{2}, \\left(\\frac{-1 + \\sqrt{5}}{2}\\right)^2\\right)$ and $B\\left(\\frac{-1 - \\sqrt{5}}{2}, \\left(\\frac{-1 - \\sqrt{5}}{2}\\right)^2\\right)$.\n\nUsing the distance formula, the distance between points $A$ and $B$ is:\n\n$$d = \\sqrt{\\left(\\frac{-1 + \\sqrt{5}}{2} - \\frac{-1 - \\sqrt{5}}{2}\\right)^2 + \\left(\\left(\\frac{-1 + \\sqrt{5}}{2}\\right)^2 - \\le

In [17]:
solution[0][1]

['To find the intersection points, we will first substitute $y=x^2$ into $x+y=1$:\n\n$ x + x^2 = 1$\n\nRewriting it as a quadratic equation:\n\n$ x^2 + x - 1 = 0$\n\nSolving this quadratic equation using the quadratic formula $x = \\frac{-b \\pm \\sqrt{b^2 - 4ac}}{2a}$ with $a=1$, $b=1$, and $c=-1$, we get:\n\n$x = \\frac{-1 \\pm \\sqrt{1^2 - 4(1)(-1)}}{2(1)} $\n\n$x = \\frac{-1 \\pm \\sqrt{1 + 4}}{2} $\n\n$x = \\frac{-1 \\pm \\sqrt{5}}{2} $\n\nSo, the intersection points are $(-\\frac{1 + \\sqrt{5}}{2}, \\frac{1 + \\sqrt{5}}{2})$ and $(-\\frac{1 - \\sqrt{5}}{2}, \\frac{1 - \\sqrt{5}}{2})$.']

已经问完了所有的subtask,最后问一次得到最终的答案

In [13]:
# 从M个路径里挑一个最好的来问,也可以问完之后再评估一下选最好的答案

user_q = f"""There is a math_problem:\n{question}

I have broken this math problem down into a series of smaller problems and each sub-problem is solved.
The sub-problems and their corresponding answers are as follows. (Format: Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx.)"""

for index, value in enumerate(solution[0][1]):  # 这里仅仅使用了最终得分最高的1条路径来总结得到final answer
    user_q += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""

Q.append({'role':'user', 'content':f"""Now that all the sub-problems have been solved, so what is the final answer?
Please give the final answer without any additional explanation or clarification."""})
finalResult = askChatGPT(Q, model=GPT_MODEL, temperature=1)

# 让大语言模型来判断有没有回答正确
judgeAnswer = {'role':'user', 'content':f"""Here is a math problem with a standard answer and a student's solution. Please help me determine if the student's solution is correct. If the numerical value are same, then it is correct.
               
Problem: {question}

Standard answer: {gold_answer}

Answer: {finalResult}

If the student's answer is correct, just output True; otherwise, just output False.
No explanation is required.
"""}

Q_judge = [judgeAnswer]
ifcorrect = askChatGPT(Q_judge, model=GPT_MODEL, temperature=1)  # 要么是True, 要么是False

if 'True' in ifcorrect:
    print('correct')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功
elif 'False' in ifcorrect:
    print('error')
    success = True  # 任务未受中断,完整地结束了,所以标记为成功                       

total tokens: 1670
total tokens: 278
error


In [14]:
print(finalResult)
print(gold_answer)

The area of the region that is common to the two triangles is \( 3\sqrt{3} \) square inches.
8\sqrt{3} \text { square inches}
